Press `shift`+`enter` to run cells

In [ ]:
%matplotlib inline

In [ ]:
import bokeh
from bokeh.plotting import figure, output_file, show, ColumnDataSource
import bokeh.models as pltm
from bokeh.io import output_notebook, show

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 250)
pd.set_option('precision', 0)


#Specific usage imports
from openfisca_core import periods, rates
import openfisca_france

In [ ]:
output_notebook()

From the `tax-benefit-system` we create a scenario (or test case) that we will use to run our simulations.

In [ ]:
tax_benefit_system = openfisca_france.FranceTaxBenefitSystem()

In [ ]:
ax_variable_max = 280000
ax_variable_min = 30_000
count = 10000

year_lag_max = 4
def make_scenario(couple = True, mar_pacs = True, nombre_enfants = 0, year = None, tax_benefit_system = tax_benefit_system,
                                    axes_variable = 'salaire_de_base',  ax_variable_max = ax_variable_max, ax_variable_min = ax_variable_min,
                                    count = count,count_span = None, age = 12, salaire_de_base_conjoint = 0):
    if mar_pacs: assert couple == True
    if count is None: count =  (ax_variable_max - ax_variable_min)/count_span #Fais une simulation par tranche de 10 euros brut
    
    enfant = [dict(age = age)]
    enfants = enfant * nombre_enfants
    
    scenario = tax_benefit_system.new_scenario().init_single_entity(
        axes = [
            [dict(count = count, 
                  min = ax_variable_min,
                  max = ax_variable_max,
                  name = axes_variable,
                  period = year-fill_lag) for fill_lag in range(year_lag_max)]
            ],
        period = year,
        parent1 = dict(
            age = 40,
            ppe_du_sa = 6020,
            statut_marital = u"pacse",
            ) if mar_pacs else dict(
            age = 40,
            ppe_du_sa = 6020, statut_marital = u"celibataire" 
            ),
        parent2 = dict(age = 40,ppe_du_sa = 6020,statut_marital = "pacse", salaire_de_base = salaire_de_base_conjoint) if mar_pacs else (dict(age = 40,ppe_du_sa = 6020,salaire_de_base = salaire_de_base_conjoint, statut_marital = u"celibataire") if couple == True else None) ,
        enfants = enfants,
        menage = dict(
            loyer = 12000,
            statut_occupation_logement = "locataire_vide",
            depcom = 13012,
            ),
        foyer_fiscal = dict(
            f7ea = nombre_enfants * ((age<15) * age>=11), #Nombre enfants à charge poursuivants leurs études : collège
            f7ec = nombre_enfants * ((age<19) * age>=15), #Nombre enfants à charge poursuivants leurs études : lycée
            f7ef = nombre_enfants * ((age<26) * age>=19), #Nombre enfants à charge poursuivants leurs études : Enseignement supérieur
            caseT = False if couple == True else True * (nombre_enfants>0)# Warning ! TODO: mieux gérer, à priori pose pas de problème si True avec un couple marié...
            ),
        )

    if (couple == True) and (mar_pacs==False):
        test_case = scenario.test_case
        test_case["foyers_fiscaux"][0]['declarants']  = ['ind0']
        test_case["foyers_fiscaux"].append({'declarants': ['ind1'],'id': 1,'caseT': False })
        scenario.test_case = test_case
        return scenario
    else:
        return scenario
        
        


In [ ]:
simulation_dict = dict()
for nb_enf in range(0,7):
    simulation_dict[nb_enf] = dict()
    for year in range(2007,2016): 
        simulation_dict[nb_enf][year] = make_scenario(couple = True,mar_pacs=True,nombre_enfants=nb_enf,year = year, count= 1000).new_simulation()

In [ ]:
simulation_dict_single = dict()
for nb_enf in range(0,6):
    simulation_dict_single[nb_enf] = dict()
    for year in range(2010,2016): 
        simulation_dict_single[nb_enf][year] = make_scenario(couple = False, mar_pacs=False, nombre_enfants=nb_enf,year = year, count=1000).new_simulation()

In [ ]:
smooth = 2

In [ ]:
source_dict_2013 = dict()
for child in range(6):
    avantage_qf_2011 = simulation_dict[child][2011].calculate("avantage_qf", period = 2011)
    avantage_qf_2013 = simulation_dict[child][2013].calculate("avantage_qf", period = 2013)
    reform_cost = avantage_qf_2013 - avantage_qf_2011
    revdisp = simulation_dict[child][2013].calculate("revenu_disponible", period = 2013)
    

    
    share_of_disposable_income = reform_cost/revdisp
    
    rni_smooth = simulation_dict[child][2011].calculate_add("rni",2011)[::smooth]
    irpp_smooth = -simulation_dict[child][2011].calculate_add("irpp",2011)[::smooth]
    mtr_2011 = rates.marginal_rate(irpp_smooth,rni_smooth)
    mtr_2011 = np.repeat(np.insert(mtr_2011, 0, 0),smooth)
    
    rni_smooth = simulation_dict[child][2014].calculate_add("rni",2014)[::smooth]
    irpp_smooth = -simulation_dict[child][2014].calculate_add("irpp",2014)[::smooth]
    mtr_2014 = rates.marginal_rate(irpp_smooth,rni_smooth)
    mtr_2014 = np.repeat(np.insert(mtr_2014, 0, 0),smooth)
    
    df = pd.DataFrame({'rbg':simulation_dict[child][2013].calculate("rbg", period = 2013),
                       'rni':simulation_dict[child][2013].calculate("rni", period = 2013),
                       'irpp': - simulation_dict[child][2013].calculate("irpp", period = 2013),
                       'decote':  simulation_dict[child][2013].calculate("decote_gain_fiscal", period = 2013),
                       'avantage_qf':simulation_dict[child][2013].calculate("avantage_qf", period = 2013),
                       'avantage_qf_2013':simulation_dict[child][2013].calculate("avantage_qf", period = 2013),
                       'child_number':simulation_dict[child][2013].calculate_add("nb_pac", period = 2013),
                       'reform_cost': reform_cost,
                       'mtr_2011' : 1-mtr_2011,
                       'mtr_2014' : 1-mtr_2014,
                       'share_of_D_income': share_of_disposable_income*100,
                      })
    source = ColumnDataSource(data=df)
    source_dict_2013[child] = source

In [ ]:
source_dict_2011 = dict()
for child in range(6):
    avantage_qf_2011 = simulation_dict[child][2011].calculate("avantage_qf", period = 2011)
    avantage_qf_2013 = simulation_dict[child][2013].calculate("avantage_qf", period = 2013)
    reform_cost = avantage_qf_2011 - avantage_qf_2013
    revdisp = simulation_dict[child][2013].calculate("revenu_disponible", period = 2013)
    

    
    share_of_disposable_income = reform_cost/revdisp
    
    revenu_disponible_smooth = simulation_dict[child][2011].calculate_add("revenu_disponible",2011)[::smooth]
    rni_smooth = simulation_dict[child][2011].calculate_add("rni",2011)[::smooth]
    irpp_smooth = -simulation_dict[child][2011].calculate_add("irpp",2011)[::smooth]
    mtr_2011 = rates.marginal_rate(irpp_smooth,rni_smooth)
    mtr_2011 = np.repeat(np.insert(mtr_2011, 0, 0),smooth)
    
    rni_smooth = simulation_dict[child][2014].calculate_add("rni",2014)[::smooth]
    irpp_smooth = -simulation_dict[child][2014].calculate_add("irpp",2014)[::smooth]
    mtr_2014 = rates.marginal_rate(irpp_smooth,rni_smooth)
    mtr_2014 = np.repeat(np.insert(mtr_2014, 0, 0),smooth)
    
    df = pd.DataFrame({'rbg':simulation_dict[child][2011].calculate("rbg", period = 2011),
                       'rni':simulation_dict[child][2011].calculate("rni", period = 2011),
                       'irpp': - simulation_dict[child][2011].calculate("irpp", period = 2011),
                       'decote':  simulation_dict[child][2011].calculate("decote_gain_fiscal", period = 2011),
                       'avantage_qf':simulation_dict[child][2011].calculate("avantage_qf", period = 2011),
                       'avantage_qf_2013':simulation_dict[child][2013].calculate("avantage_qf", period = 2013),
                       'child_number':simulation_dict[child][2013].calculate_add("nb_pac", period = 2013),
                       'reform_cost': reform_cost,
                       'mtr_2011' : 1 -mtr_2011,
                       'mtr_2014' : 1 - mtr_2014,
                       'share_of_D_income': share_of_disposable_income*100,
                      })
    source = ColumnDataSource(data=df)
    source_dict_2011[child] = source

In [ ]:

import jinja2
from bokeh.embed import components

# IMPORTANT NOTE!! The version of BokehJS loaded in the template should match 
# the version of Bokeh installed locally.

template = jinja2.Template("""
<!DOCTYPE html>
<html lang="en-US">

<link
    href="https://cdn.pydata.org/bokeh/release/bokeh-{0}.min.css"
    rel="stylesheet" type="text/css"
>
<script 
    src="https://cdn.pydata.org/bokeh/release/bokeh-{0}.min.js"
></script>

<body>

    
    {{{{ script }}}}
    
    {{{{ div }}}}

</body>

</html>
""".format(bokeh.__version__))
p = figure(width=1000, height=500, title='Hover over points')

In [ ]:
TOOLTIPS = """
    <style>
        .bk-tooltip>div:not(:first-child) {display:none;}
    </style>
    <div>
        <div class="container-fluid">
            
            <span style="font-size: 15px; font-weight: bold;">@child_number{0,0.} 👶: <br></span>
            <span style="font-size: 15px ;">   &nbsp  Income: @rbg{0,0.} € <br></span>
            <span style="font-size: 15px ;">   &nbsp  Tax: @irpp{0,0.} €</span>
            <span style="font-size: 15px ;">   &nbsp  Avantage_qf: @avantage_qf{0,0.} € <br></span>
 

            <span style="font-size: 15px; font-weight: bold;"> &nbsp  MTR:</span> <span style="font-size: 15px;"> &nbsp 2011: @mtr_2011{0,.01}, 2014: @mtr_2014{0,.01}  <br></span>

            <span style="font-size: 15px ;">   &nbsp  Reform cost: @reform_cost{0,0.} €</span>
            <span style="font-size: 15px ;">   &nbsp  Share of Disposable Income: @share_of_D_income{0,0.00}% <br></span>
            
    </div>
"""

In [ ]:
# https://bokeh.pydata.org/en/latest/docs/reference/models/tools.html
# https://bokeh.pydata.org/en/latest/docs/reference/core/enums.html#bokeh.core.enums.TooltipAttachment

In [ ]:
from bokeh.models import Range1d
from bokeh.models import NumeralTickFormatter
p = figure(width=100, height=50, title='Fiscal gain from FQ',tooltips = TOOLTIPS)

#p.add_tools(pltm.HoverTool(tooltips=TOOLTIPS, mode = "vline")
#            )



for child in range(1,4):
    red_renderer_line = p.line(x='rni', y='avantage_qf', source=source_dict_2013[child], line_color="blue", legend = "2013")
    
renderer_line_list = list()
for child in range(1,4):
    renderer_line_list.append(p.line(x='rni', y='avantage_qf', source=source_dict_2011[child], line_color="red", legend = "2011"))
p.sizing_mode = "scale_width"
p.x_range=Range1d(40_000, 100_000)
p.xaxis.formatter = NumeralTickFormatter(format=u"0")
p.yaxis.axis_label = "Fiscal gain in euros"
p.xaxis.axis_label = "Taxable income in euros"
p.legend.location = "top_left"

p.add_tools(pltm.HoverTool(tooltips=TOOLTIPS, mode = "vline", renderers=renderer_line_list)
            )
p.add_tools(pltm.HoverTool(tooltips=TOOLTIPS, mode = "vline", renderers=renderer_line_list[2:])
            )


show(p)
script, div = components(p)

In [ ]:
file2  = open("/Users/adrienpacifico/Informatique/adrienpacifico.github.io/docs/bokeh_graph.html", "w")
file2.write(template.render(script=script, div=div)) 
file2.close()
from bokeh.io import reset_output
reset_output()

In [ ]:
from IPython.display import HTML
HTML(template.render(script=script, div=div))


In [ ]:
gain = simulation_dict[2][2011].calculate("avantage_qf", period = 2011) ; print(gain.shape)
print(np.where(gain<gain[::-1])[0][-1])
simulation_dict[2][2011].calculate_add("salaire_net", period = year)[::2+2][np.where(gain<gain[::-1])[0][-1]]

In [ ]:
df = pd.DataFrame()
for year in [2011,2013]:
    s = pd.Series()
    for nb_enf in range(1,7):
        gain = simulation_dict[nb_enf][year].calculate("avantage_qf", period = year)
        s = s.append(pd.Series(simulation_dict[nb_enf][year].calculate_add("salaire_net", period = year)[::2+nb_enf][np.where(gain<gain[-1])[0][-1]]))
    df[year] =s 
#df.reset_index(inplace = True, drop = True);df.index.name = "Nb child" ;
df.index = list(range(1,7));df.index.name = "Children #"

In [ ]:
df

In [ ]:
print(df.to_html())

In [ ]:
df = pd.DataFrame()
for year in [2011,2013]:
    s = pd.Series()
    for nb_enf in range(1,7):
        gain = simulation_dict[nb_enf][year].calculate("avantage_qf", period = year)
        s = s.append(pd.Series(simulation_dict[nb_enf][year].calculate_add("rbg", period = year)[np.where(gain<gain[-1])[0][-1]]))
    df[year] =s 
#df.reset_index(inplace = True, drop = True);df.index.name = "Nb child" ;
df.index = list(range(1,7));df.index.name = "Children #"

In [ ]:
df

In [ ]:
print(df.to_html())